# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wynyard,-41.0000,145.7167,14.88,92,100,5.33,AU,1702505621
1,1,east london,-33.0153,27.9116,20.13,89,97,7.26,ZA,1702505621
2,2,isafjordur,66.0755,-23.1240,1.07,87,57,4.35,IS,1702505591
3,3,okoneshnikovo,54.8377,75.0834,-19.13,80,100,5.91,RU,1702505622
4,4,iqaluit,63.7506,-68.5145,-13.15,72,75,3.09,CA,1702505579


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df[(city_data_df['Max Temp']>24) & (city_data_df['Max Temp']<35) & (city_data_df['Cloudiness'] < 50)]

# Drop any rows with null values
ideal_city.dropna()

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bahia de kino,28.8333,-111.9333,25.96,48,0,3.16,MX,1702505623
16,16,inarajan village,13.2736,144.7484,28.54,86,27,7.85,GU,1702505627
30,30,port mathurin,-19.6833,63.4167,26.31,74,7,4.90,MU,1702505601
32,32,georgetown,5.4112,100.3354,25.96,89,20,3.60,MY,1702505627
47,47,nadi,-17.8000,177.4167,30.00,45,20,1.54,FJ,1702505586
81,81,seven pagodas,12.6264,80.1722,24.03,75,7,5.52,IN,1702505650
87,87,hawaiian paradise park,19.5933,-154.9731,26.73,76,40,2.06,US,1702505652
93,93,burutu,5.3509,5.5076,25.51,58,6,1.64,NG,1702505656
119,119,cabo san lucas,22.8909,-109.9124,27.64,43,40,2.57,MX,1702505503
120,120,moranbah,-22.0016,148.0466,25.50,66,46,6.42,AU,1702505668


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns = ['City','Country','Lat','Lng','Humidity']
hotel_df = ideal_city[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bahia de kino,MX,28.8333,-111.9333,48,
16,inarajan village,GU,13.2736,144.7484,86,
30,port mathurin,MU,-19.6833,63.4167,74,
32,georgetown,MY,5.4112,100.3354,89,
47,nadi,FJ,-17.8000,177.4167,45,
81,seven pagodas,IN,12.6264,80.1722,75,
87,hawaiian paradise park,US,19.5933,-154.9731,76,
93,burutu,NG,5.3509,5.5076,58,
119,cabo san lucas,MX,22.8909,-109.9124,43,
120,moranbah,AU,-22.0016,148.0466,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # name_address
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    
    # print(f"{name_address}")
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bahia de kino - nearest hotel: Hotel Hacienda
inarajan village - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
georgetown - nearest hotel: Page 63 hostel
nadi - nearest hotel: President Hotel
seven pagodas - nearest hotel: Radisson Temple Bay
hawaiian paradise park - nearest hotel: No hotel found
burutu - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
moranbah - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
kingston - nearest hotel: Aphrodite Hotel
carnarvon - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
la gomera - nearest hotel: No hotel found
puerto el triunfo - nearest hotel: Motel Jardín
paracale - nearest hotel: Malayan Tourist Inn
kailua-kona - nearest hotel: Kona Seaside Hotel
itamaraca - nearest hotel: No hotel found
maceio - nearest hotel: Maceió Hostel e Pousada
utrik - nearest hotel: No hotel found
pitimbu - nearest hotel: Pousada Br

,City,Country,Lat,Lng,Humidity,Hotel Name
5,bahia de kino,MX,28.8333,-111.9333,48,Hotel Hacienda
16,inarajan village,GU,13.2736,144.7484,86,No hotel found
30,port mathurin,MU,-19.6833,63.4167,74,Escale Vacances
32,georgetown,MY,5.4112,100.3354,89,Page 63 hostel
47,nadi,FJ,-17.8000,177.4167,45,President Hotel
81,seven pagodas,IN,12.6264,80.1722,75,Radisson Temple Bay
87,hawaiian paradise park,US,19.5933,-154.9731,76,No hotel found
93,burutu,NG,5.3509,5.5076,58,No hotel found
119,cabo san lucas,MX,22.8909,-109.9124,43,Comfort Rooms
120,moranbah,AU,-22.0016,148.0466,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ['Hotel Name','Country']
)



# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)